# 搭建读取化学文献的rag模型

## step1 导入相关包

In [1]:
import os
# import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

print("Starting..........")
os.environ['OPENAI_API_KEY'] ="ab78b1a21a614ba1ade36ba88b172893"
os.environ['OPENAI_BASE_URL'] ="https://api.lingyiwanwu.com/v1"
print("Loading.....")

Starting..........
Loading.....


## step2 加载文档

In [5]:
root_dir = "./papers"

def extract_file_dirs(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                fp = os.path.join(root, file)
                file_paths.append(fp)
    return file_paths

files = extract_file_dirs(root_dir)
files

['./papers\\Enantioselective Iridium-Catalyzed Allylic Substitution with 2-Methylpyridines.pdf',
 './papers\\Iridium-Catalyzed Asymmetric Allylic Amination Reaction with N-Aryl Phosphoramidite Ligands.pdf']

In [6]:
docs = []
for file_path in files:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())
docs
print("Loadded....")

Loadded....


## step3 文档分割

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits  = []
splits.extend(text_splitter.split_documents(docs))
splits, len(splits)

([Document(metadata={'source': './papers\\Enantioselective Iridium-Catalyzed Allylic Substitution with 2-Methylpyridines.pdf', 'page': 0}, page_content='German Edition: DOI: 10.1002/ange.201700433Asymmetric CatalysisInternational Edition: DOI: 10.1002/anie.201700433\nEnantioselective Iridium-Catalyzed Allylic Substitution with\n2-Methylpyridines\nXi-Jia Liu and Shu-Li You*\nAbstract: An enantioselective iridium-catalyzed allylic sub-\nstitution with a set of highly unstabilized nucleophiles gen-\nerated in situ from 2-methylpyridines is described. Enantioen-'),
  Document(metadata={'source': './papers\\Enantioselective Iridium-Catalyzed Allylic Substitution with 2-Methylpyridines.pdf', 'page': 0}, page_content='riched 2-substituted pyridines, which are frequently encoun-tered in natural products and pharmaceuticals, could be easilyconstructed by this simple method in good yields and excellentenantioselectivity. The synthetic utility of the pyridine productsis demonstrated through the s

## step4 向量模型及向量数据库

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="shibing624/text2vec-base-multilingual")

# 保存到当前根目录的vector_db文件夹下
kb_name = "vector_db_1"
persist_directory = os.path.join(os.getcwd(), kb_name)

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory,
)
vectordb.persist()

c:\Users\HHH\.conda\envs\chemical_rag\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\HHH\.conda\envs\chemical_rag\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\HHH\.conda\envs\chemical_rag\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen

In [11]:
retriever = vectordb.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
print("RAG....")

LangSmithUserError: API key must be provided when using hosted LangSmith API

## step5 构建模型和链

## step6 检索